In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
pages = set()
def getLinks(url):
    global pages
    html = urlopen("https://sg.indeed.com/jobs?q=software+engineer&l=Singapore" + url)
    bsObj = BeautifulSoup (html)
    for link in bsObj.findAll("a", href = re.compile("^(/jobs?)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                pages.add(newPage + '\n')
                getLinks(newPage)
                
getLinks("")
with open('C:/Users/Phu Wai Paing/Desktop/Course Books/Data Extraction/UserTest/urlLinks.txt', 'w')as f:
    f.writelines(pages)

In [2]:
!pip install selenium

In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time


driver = webdriver.Firefox(executable_path = "C:/Users/Phu Wai Paing/Desktop/Course Books/Data Extraction/geckodriver/geckodriver.exe")

with open('C:/Users/Phu Wai Paing/Desktop/Course Books/Data Extraction/UserTest/urlLinks.txt', 'r') as text:
    links = text.read().splitlines()
count = 0
for url in links:
    driver.get("https://sg.indeed.com" + url)
    btn = driver.find_elements_by_class_name("clickcard")
    for i in range(0,len(btn)):
        count += 1
        btn[i].click()
        time.sleep(5)
        with open("C:/Users/Phu Wai Paing/Desktop/Course Books/Data Extraction/UserTest/ExtractData/text %d.txt" % count, "w+", encoding='utf-8') as f:
            f.write(driver.find_element_by_id("vjs-desc").text)

KeyboardInterrupt: 

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import re

def read_text(filename):
    file = open(filename, "r", encoding="utf-8")
    filedata1 = file.read()
    r = re.findall("(Salary\s*\w*: .+)", filedata1)
    if not r:
        ne = 'Negotiate'
        f.write(ne + "*")
    else:
        f.write("".join(r) + "*")
    #print(filedata)
    file1 = open(filename, "r", encoding="utf-8")
    filedata = file1.readlines()
    sentences = []
    for i in filedata:
        article = i.split("\n")
        #print(article)
        for sentence in article:
            #print(sentence)
            sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()
    file.close()
    return sentences
    


def sentence_similarity(sent1, sent2, stopwords = None):
    if stopwords is None:
        stopwords = []
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    all_words = list(set(sent1 + sent2))
    
    vector1 = [0]*len(all_words)
    vector2 = [0]*len(all_words)
    #build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
        
    #build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
    return 1-cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    #Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

def generate_summary(filename, top_n = 10):
    stop_words = stopwords.words('english')
    summarize_text = []
    
    #step1 - read text and split it
    sentences = read_text(filename)

    #step2 - generate similarity matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)
    
    #step3 - rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    #print(sentence_similarity_graph)
    scores = nx.pagerank(sentence_similarity_graph)

    #step 4 - sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    #print("Indexes of top ranked_sentence order are", ranked_sentence)
    
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
        
    #step 5 - output the summeraize text
    print("Summarize Text: \n", ". ". join(summarize_text))
    f.write("".join(summarize_text) + "\n")
    

In [3]:
import os
directory = 'C:/Users/Phu Wai Paing/Desktop/Course Books/Data Extraction/UserTest/ExtractData/'
f = open ("result.txt", "w+", encoding="utf-8")
f.write("Salary*Summary" + "\n")
for filename in os.listdir(directory):
    print(filename)
    generate_summary(directory + filename, 3)
f.close()

text 1.txt
Summarize Text: 
 . . 
text 10.txt
Summarize Text: 
 At least 1 year of experience in one of the following Software Development / Algorithms Development / Machine Learning. At least 1 year of experience in one of the following Software Development / Algorithms Development / Machine Learning. Proficient in the following Python libraries
text 100.txt
Summarize Text: 
 Hands-on web services development in Golang.. Grab operates a multi-sided marketplace for passengers, drivers, and merchants. Pricing works as an essential economic lever to facilitate the transaction for all players. The objective of the pricing team is to set the optimal price to deliver value (efficiency, sustainability, delightful experiences) to Grab and our customers.. In this exciting opportunity, you would be laying down a multi-year strategy to improve the reliability and efficiency of our pricing systems to achieve 99.99+ availability. You would also be inheriting a highly functional engineering team as

text 27.txt
Summarize Text: 
 As a System Characterization Product Engineer in the Non-Volatile Engineering (NVE) group at Micron Technology, Inc, you will be responsible for developing and maintaining system testing for characterization and validation of Micron’s leading-edge next generation non-volatile memory devices. You will develop test programs, perform failure analysis, and analyze data to ensure Micron’s devices meet internal and external customers’ specifications. The tests you develop will provide critical information and characterization data to engineering teams throughout the company for product development and mass production.. Develop test programs in C/C++ for system characterization. Excellent knowledge and experience in C/C++ programming
text 28.txt
Summarize Text: 
 As a Data Scientist at Micron Smart manufacturing and Artificial Intelligence organization, you will be working multidisciplinary projects in the different area which leveraging big data in Micron and Su

Summarize Text: 
 At least 3 years of relevant work experience;. Unix, AIX, Solaris, or Redhat;: 1 year (Preferred). We’re looking for a System Engineer who is able to manage systems and infrastructure, and to plan and lead system design, configuration, administration and troubleshooting.
text 5.txt
Summarize Text: 
 The term of appointment shall be up to 24 months, with the possibility of extension. The remuneration and benefits are internationally competitive, and commensurate with qualifications and experience.. The Research Engineer (RE) will be responsible for investigating stormwater quality modeling aspects of the project and also assist the senior researchers in data collection and analysis. The candidate is expected to have good experience in water quality assessment and in hydrological and water quality modelling and exposure to MATLAB/PYTHON.. Applicants should have a BEng or Masters in Civil/Environmental/Chemical Engineering or equivalent with background knowledge in model

Summarize Text: 
 At FunctionEight, we want you to love your job so we’ve created a fun working environment with plenty of opportunities to socialise. We’re also eager to reward you for your hard work and to help you grow. Oh, and we encourage you to have a life outside of work.. At least 3-4 years working experience in a Technical Support role.. Install and configure any software solutions that the client is requesting.
text 7.txt
Summarize Text: 
 Our Mission – Securing the software that powers your world. At Veracode, we are focused on that mission every day. Veracode is recognized as a premier provider of SaaS-based application security solutions, transforming the way companies secure applications in today's software driven world. We provide our customers with a solid foundation on which to build security into their modern agile development processes.. Other activities that involve security research about software vulnerabilities. Preferable if the candidate has knowledge of softwa

Summarize Text: 
 Develop and implement new test concepts and methods, programs, test jigs, fixtures and test equipment for new products testing which includes Optical.. Develop test strategy for new products and new techniques for test handling and fixtures, using experimental design and statistical method, evaluate failure data to find root cause and implement sound engineering solution. Ability to develop simple optical test bench, starting from the test specifications. Choose instruments/equipment, decide how to connect and interface them and write the test program
text 84.txt
Summarize Text: 
 Responsible for recombination mastering development. Development of the mastering process for new application. Basic knowledge on polydimethylsiloxane (PDMS) molding as well as UV & thermal curing epoxy will be an advantage
text 85.txt
Summarize Text: 
 Experience in CVD module with hands-on experience for Nitride/Oxide process types. Troubleshoot process deviation and equipment malfunctions

In [4]:
import pandas as pd

df = pd.read_csv("result.txt", sep = "*")
df.to_csv('Result_csv.csv')